In [4]:
from sklearn.model_selection import train_test_split
from pathlib import Path
import matplotlib.image as img
import json
import numpy as np
from tqdm import tqdm
import h5py

from multiprocessing import Pool
import time
from concurrent.futures import ThreadPoolExecutor
import functools
import pandas as pd

In [5]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [7]:
# Helper Functions from EDA
def ejson(p, fn): # extract json
    with open((p/fn).as_posix()) as f: return json.load(f) 
## Simple Json Reading Functions
def info_data(p): return ejson(p, 'info.json')
def dot_data(p): return ejson(p, 'dotInfo.json')
def frame_data(p): return ejson(p, 'frames.json')    
def screen_data(p): return ejson(p, 'screen.json')
def get_frame(p, img_fn): return img.imread(p/'..'/'..'/'gazecapture-224x224'/p.name/'frames'/img_fn)
## Larger Helper Functions
def coordinate_ydata(p): 
    data = dot_data(p)
    return data['YCam'] # we want relative to camera coords
def coordinate_xdata(p): 
    data = dot_data(p)
    return data['XCam'] # we want relative to camera coords

In [13]:
dset_path = Path('/media/brennan/Data/ml/gazecapture/')

# Export to hdf5
Not Used - See below for better soln.

In [9]:
# Extracting and storing X values
def extract_photo(f, case): return get_frame(case, f)

def extract(case):
    F = frame_data(case) # list of file names correlated to camera images 
    with Pool(6) as p:  # multi core usale
        x = p.map(functools.partial(extract_photo, case=case), F)
    return x

def extract_to_hdf5(cname, bs, extract_fcn): # bs dsets per hp5y file
    f = h5py.File('{}-0.hdf5'.format(cname), 'w') 
    for i, case in enumerate(list(dset_path.iterdir())):
        cn = case.name
        edata = extract_fcn(case)
        
        dset = f.create_dataset('dset'+cn, (len(edata), 224, 224, 3))
        dset[...] = edata

        if i % bs == 0 and i!= 0:
            f.close()
            f = h5py.File('{}-{}.hdf5'.format(cname, i//bs), 'w')

In [2]:
extract_to_hdf5('X', 500, extract)

In [ ]:
extract_to_hdf5('Y_Xpoint', 2000, coordinate_xdata)
extract_to_hdf5('Y_YPoint', 2000, coordinate_ydata)

# Extract File Names For Keras Generator

In [15]:
# split into train / test sets
# then split the train set into train / val sets
cases = []
fnames = []
for case in tqdm(list(dset_path.iterdir())):
    F = frame_data(case)
    for f in F:
        cases.append(case.name)
        fnames.append(f)

100%|██████████| 1474/1474 [00:04<00:00, 317.96it/s]


In [16]:
# create dataframe from data
df = pd.DataFrame(data={'case_name': cases, 'file_names': fnames})
df.head()

,case_name,file_names
0,00002,00000.jpg
1,00002,00001.jpg
2,00002,00002.jpg
3,00002,00003.jpg
4,00002,00004.jpg


In [17]:
test_size = 0.1
train, test = train_test_split(df, test_size=test_size)

In [18]:
train.shape, test.shape

((2200953, 2), (244551, 2))

In [ ]:
f = h5py.File('X-0.hdf5', 'r')
for k in list(f.keys()):
    print(f[k].shape)

In [22]:
train.to_csv('train-df.csv')

In [23]:
test.to_csv('test-df.csv')